In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-05"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
13491,2024-05-05,Japão B.League,02:05,Yokohama,Brave Thunders,3.98,1.22,159.5,1.85,1.85,0.0,0.00,0.00,n92LipyD,0.251256,0.819672,0.540541,0.540541,0.070928,193.032,74.631699,0.386629,0.6,1.341641,2.236068,304.52,2.794,0.980423,0.350903,-74.0,131.460,37.477196,0.285084,2.4,1.341641,0.559017,127.40,1.604,0.282454,0.176093,26.0,92,98,3.31,1.30,249.980,133.706,0.750621,0.000000,NaN,-5.00,-1.000,-2.980000,0.477623,0.2,-0.277623,1.10,0.220,1.000000,0.649499,0.6,-0.049499
13492,2024-05-05,Japão B.League,03:05,Gunma,Sendai,1.53,2.39,162.5,1.80,1.90,0.0,0.00,0.00,vqaDg660,0.653595,0.418410,0.555556,0.526316,0.072005,302.320,248.489540,0.821942,1.8,1.643168,0.912871,153.26,3.690,3.033892,0.822193,22.0,193.014,139.115652,0.720754,1.2,1.643168,1.369306,194.65,2.412,1.922374,0.797004,3.0,97,85,1.58,2.29,127.274,240.240,0.310261,0.038222,NaN,0.67,0.134,3.955224,0.641154,0.6,-0.041154,0.26,0.052,26.730769,0.412679,0.4,-0.012679
13493,2024-05-05,Japão B.League,03:05,Ibaraki Robots,Utsunomiya Brex,6.04,1.10,157.5,1.83,1.87,0.0,0.00,0.00,dK98fnif,0.165563,0.909091,0.546448,0.534759,0.074654,257.854,60.197955,0.233458,1.2,1.643168,1.369306,328.04,3.436,1.341764,0.390502,-46.0,116.934,16.673241,0.142587,3.0,0.000000,0.000000,112.00,1.376,0.239019,0.173706,89.0,59,100,5.56,1.12,243.914,106.998,0.978461,0.015289,NaN,-1.00,-0.200,-25.200000,0.303924,0.3,-0.003924,0.32,0.064,1.562500,0.783842,0.9,0.116158
13494,2024-05-05,Japão B.League,03:05,Kyoto,Nagasaki,2.20,1.62,164.5,1.80,1.90,-2.5,1.98,1.82,t8qCE7rQ,0.454545,0.617284,0.555556,0.526316,0.071829,240.530,81.883486,0.340429,1.8,1.643168,0.912871,271.81,2.860,1.015357,0.355020,-1.0,171.416,56.137131,0.327491,1.8,1.643168,0.912871,125.16,2.170,0.903521,0.416369,3.0,77,84,3.53,1.49,215.674,153.374,0.214724,0.038222,0.059546,-0.51,-0.102,-11.764706,0.392189,0.2,-0.192189,0.36,0.072,8.611111,0.439478,0.4,-0.039478
13495,2024-05-05,Lituânia Lkl,11:20,BC Wolves,Zalgiris Kaunas,3.20,1.32,163.5,1.85,1.85,0.0,0.00,0.00,fPxOPHXk,0.312500,0.757576,0.540541,0.540541,0.070076,118.638,6.777796,0.057130,3.0,0.000000,0.000000,129.00,1.216,0.192172,0.158036,104.0,111.384,13.491445,0.121126,2.4,1.341641,0.559017,104.04,1.156,0.131453,0.113714,69.0,86,102,1.50,1.02,128.640,111.264,0.588213,0.000000,NaN,1.11,0.222,9.909910,0.795326,0.7,-0.095326,-0.37,-0.074,-4.324324,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13604,2024-05-05,Austrália Nbl1 Norte,02:00,Southern District,Brisbane Capitals,1.58,2.29,170.5,1.85,1.85,0.0,0.00,0.00,Uur98Nu5,0.632911,0.436681,0.540541,0.540541,0.069593,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,203.224,104.555346,0.514483,0.6,1.341641,2.236068,128.65,2.368,1.149248,0.485324,-30.0,73,83,1.54,1.55,0.000,203.224,0.259455,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,-3.47,-0.694,-1.858790,0.000000,0.0,0.000000
13605,2024-05-05,Austrália Nbl1 Norte,01:00,South West Metro Pirates,Rockhampton Rockets,1.30,3.31,185.5,1.85,1.85,0.0,0.00,0.00,zJr593Qb,0.769231,0.302115,0.540541,0.540541,0.071346,0.000,0.000000,0.000000,0.0,0.

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
13526,13:00,Polônia 1. Liga,Gornik Walbrzych,Starogard Gdanski,1.61,Back Home
